# Northern Atlantic Oscillation

In [1]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, addcyclic, shiftgrid
import pymongo
from pprint import pprint
from datetime import datetime, timedelta, date
import pandas as pd
import gdal as gdl
import matplotlib.mlab as ml
import cartopy.crs as ccrs

In [2]:
mongo_host_local = 'mongodb://localhost:27017/'
mg = pymongo.MongoClient(mongo_host_local)

In [3]:
db = mg.ECMWF
db.collection_names()

['ERAINT_grid',
 'ERAINT_lores_grid',
 'ERAINT_lores_monthly',
 'ERAINT_lores_monthly_anom',
 'ERAINT_monthly',
 'system.indexes']

In [6]:
ERA_vers = 'lores'
if (ERA_vers == 'hires'):
    col_dat = 'ERAINT_monthly'
    col_anom = 'ERAINT_monthly_anom'
    col_grid = 'ERAINT_grid'
    resolution = 0.25
elif (ERA_vers == 'lores'):
    col_dat = 'ERAINT_lores_monthly'
    col_anom = 'ERAINT_lores_monthly_anom'
    col_grid = 'ERAINT_lores_grid'
    resolution = 2.5

# Query grid cell ids for the NAO regions

"The NAO index is defined here as the difference between the SLP anomalies averaged over the domains of (50°W–10°E, 25–55°N) and (40°W–20°E, 55–85°N). This is referred as the “ERA-Interim Domain-based” definition." (see Wang et al., 2017)

In [7]:
con_grid = db[col_grid]
fo = con_grid.find_one()
pprint(fo)

{'_id': ObjectId('5a1882228cb6b815bce38f92'),
 'id_grid': 1,
 'loc': {'coordinates': [-180.0, 90.0], 'type': 'Point'}}


In [18]:
this_polygon = [list(reversed([ [-50,25], [-50,55], [10,55],[ 10,25], [-50,25]])), 
                list(reversed([ [-40, 55], [-40, 85], [20, 85], [20, 55], [-40, 55]]))]
this_polygon

[[[-50, 25], [10, 25], [10, 55], [-50, 55], [-50, 25]],
 [[-40, 55], [20, 55], [20, 85], [-40, 85], [-40, 55]]]

In [31]:
poly1 = [list(reversed([ [-50,25], [-50,55], [10,55],[ 10,25], [-50,25]]))]
poly2 = [list(reversed([ [-40, 55], [-40, 85], [20, 85], [20, 55], [-40, 55]]))]
def getGridIds(this_polygon):
    geo_qry = {"loc": 
               {"$geoWithin": {
                   "$geometry": {
                       "type": "Polygon",
                       "coordinates": this_polygon
                   }
               }}}

    res = con_grid.find(filter = geo_qry, projection = {"_id":0, "id_grid": 1, "loc": 1})
    grid_df = pd.DataFrame(list(res))
    return grid_df
grid_df1 = getGridIds(poly1)
grid_ids1 = grid_df1.id_grid.values
grid_df2 = getGridIds(poly2)
grid_ids2 = grid_df2.id_grid.values
len(grid_ids1)

306